# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [1]:
import numpy as np
import pandas as pd



# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:

pokemon.head()


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [9]:
count = pokemon.groupby(["Legendary"]).count()


count

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
Legendary,,,,,,,,,,,,
False,735,735,735,374,735,735,735,735,735,735,735,735
True,65,65,65,40,65,65,65,65,65,65,65,65


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [13]:
mean_points_leg = pokemon[pokemon["Legendary"] == True]["Total"].mean()

std_points_leg = pokemon[pokemon["Legendary"] == True]["Total"].std()

print(mean_points_leg, std_points_leg)

637.3846153846154 60.93738905315346


In [15]:
mean_points_non_leg = pokemon[pokemon["Legendary"] == False]["Total"].mean()

std_points_non_leg = pokemon[pokemon["Legendary"] == False]["Total"].std()

print(mean_points_non_leg, std_points_non_leg)

417.21360544217686 106.76041745713022


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [18]:
from scipy.stats import ttest_ind
import scipy.stats as st

# H0: mu leg = mu non leg
# H1: mu leg =! mu non leg

leg = pokemon[pokemon["Legendary"] == True]["Total"]

non_leg = pokemon[pokemon["Legendary"] == False]["Total"]

st.ttest_ind(leg, non_leg, equal_var = False)



Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957446e-47)

What do you conclude from this test? Write your conclusions below.

In [ ]:
# Your conclusions here:


# p value is low, so we reject that the measn of the 2 groups is equal.

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [22]:
count_types = pokemon.groupby(["Type 1"]).count()["Name"]

count_types


Type 1
Bug          69
Dark         31
Dragon       32
Electric     44
Fairy        17
Fighting     27
Fire         52
Flying        4
Ghost        32
Grass        70
Ground       32
Ice          24
Normal       98
Poison       28
Psychic      57
Rock         44
Steel        27
Water       112
Name: Name, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [26]:
# H0: mean water == to other pokemons
# H1 mean water != to other pokemons

water = pokemon[pokemon["Type 1"] == "Water"].mean()

non_water = pokemon[pokemon["Type 1"] != "Water"].mean()

st.ttest_ind(water, non_water, equal_var = False)

## H0 rejected: the mean is not equal


/var/folders/ms/vrwr3_md2xl156mcvfmyskx80000gn/T/ipykernel_22690/886052578.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  water = pokemon[pokemon["Type 1"] == "Water"].mean()
/var/folders/ms/vrwr3_md2xl156mcvfmyskx80000gn/T/ipykernel_22690/886052578.py:6: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  non_water = pokemon[pokemon["Type 1"] != "Water"].mean()


Ttest_indResult(statistic=-0.12495908786675247, pvalue=0.9019514128217778)

In [27]:
# H0: std water == to other pokemons
# H1 std water != to other pokemons

water = pokemon[pokemon["Type 1"] == "Water"].std()

non_water = pokemon[pokemon["Type 1"] != "Water"].std()

st.ttest_ind(water, non_water, equal_var = False)

## H0 rejected: the std is not equal

/var/folders/ms/vrwr3_md2xl156mcvfmyskx80000gn/T/ipykernel_22690/1609278108.py:4: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  water = pokemon[pokemon["Type 1"] == "Water"].std()
/var/folders/ms/vrwr3_md2xl156mcvfmyskx80000gn/T/ipykernel_22690/1609278108.py:6: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  non_water = pokemon[pokemon["Type 1"] != "Water"].std()


Ttest_indResult(statistic=-0.16760510038991525, pvalue=0.868781033307666)

Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [30]:
# H0: total point of water == to other pokemons
# H1 total points of water != to other pokemons

water = pokemon[pokemon["Type 1"] == "Water"]["Total"]

non_water = pokemon[pokemon["Type 1"] != "Water"]["Total"]

st.ttest_ind(water, non_water, equal_var = False)

## H0 not rejected: total points is equal



Ttest_indResult(statistic=-0.4638681676327303, pvalue=0.6433915385821449)

Write your conclusion below.

In [ ]:
# Your conclusions here:



# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [34]:
# H0: pokemens defense scores == to attck scores
# H1: pokemens defense scores != to attck scores

defense = pokemon["Defense"]

attack = pokemon["Attack"]



st.ttest_rel(defense, attack)



TtestResult(statistic=-4.325566393330478, pvalue=1.7140303479358558e-05, df=799)

Describe the results of the test in the cell below.

In [ ]:
## p value is very low, so we have to reject H0, so defense scores are very different
# from attack scores



We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [35]:
# H0: pokemens mean speaicl def == mean special attack 
# H1: pokemens mean speaicl def == mean special attack 

sp_attack = pokemon["Sp. Atk"].mean()

sp_defense = pokemon["Sp. Def"].mean()


st.ttest_rel(sp_attack, sp_defense)



/Users/marianasousa/anaconda3/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:502: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/marianasousa/anaconda3/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/marianasousa/anaconda3/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


TtestResult(statistic=nan, pvalue=nan, df=0)

Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:

## do not know why p_value is nan

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [38]:
# H0: difference between defense mean and attack mean = 0
# H0: difference between defense mean and attack mean != 0


difference = pokemon["Defense"] - pokemon["Attack"]


st.ttest_1samp(difference, 0)


## p value is veeeeeery low, so the diference between defense and attack is different than 0.
    

TtestResult(statistic=-4.325566393330478, pvalue=1.7140303479358558e-05, df=799)

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [ ]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [ ]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# Your answer here:

